# 状态
亦称：State  
状态是一种行为设计模式，让你能在一个对象的内部状态变化时改变其行为，使其看上去就像改变了自身所属的类一样。

## Keyword
Context类可以看做整体控制，包含状态转化与功能请求。抽象State类实现Context的属性化访问与修改以及设置抽象逻辑功能，具体State类实现各自需要的功能。

## 最佳抽象

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod


class Context:
    """
    The Context defines the interface of interest to clients. It also maintains
    a reference to an instance of a State subclass, which represents the current
    state of the Context.
    """

    _state = None
    """
    A reference to the current state of the Context.
    """

    def __init__(self, state: State) -> None:
        self.transition_to(state)

    def transition_to(self, state: State):
        """
        The Context allows changing the State object at runtime.
        """

        print(f"Context: Transition to {type(state).__name__}")
        self._state = state
        self._state.context = self

    """
    The Context delegates part of its behavior to the current State object.
    """

    def request1(self):
        self._state.handle1()

    def request2(self):
        self._state.handle2()


class State(ABC):
    """
    The base State class declares methods that all Concrete State should
    implement and also provides a backreference to the Context object,
    associated with the State. This backreference can be used by States to
    transition the Context to another State.
    """

    @property
    def context(self) -> Context:
        return self._context

    @context.setter
    def context(self, context: Context) -> None:
        self._context = context

    @abstractmethod
    def handle1(self) -> None:
        pass

    @abstractmethod
    def handle2(self) -> None:
        pass


"""
Concrete States implement various behaviors, associated with a state of the
Context.
"""


class ConcreteStateA(State):
    def handle1(self) -> None:
        print("ConcreteStateA handles request1.")
        print("ConcreteStateA wants to change the state of the context.")
        self.context.transition_to(ConcreteStateB())

    def handle2(self) -> None:
        print("ConcreteStateA handles request2.")


class ConcreteStateB(State):
    def handle1(self) -> None:
        print("ConcreteStateB handles request1.")

    def handle2(self) -> None:
        print("ConcreteStateB handles request2.")
        print("ConcreteStateB wants to change the state of the context.")
        self.context.transition_to(ConcreteStateA())


if __name__ == "__main__":
    # The client code.

    context = Context(ConcreteStateA())
    context.request1()
    context.request2()

Context: Transition to ConcreteStateA
ConcreteStateA handles request1.
ConcreteStateA wants to change the state of the context.
Context: Transition to ConcreteStateB
ConcreteStateB handles request2.
ConcreteStateB wants to change the state of the context.
Context: Transition to ConcreteStateA


## 最简实践
编写两个状态来模拟英雄联盟的双形态英雄杰斯，Context类作为总的控制，实现形态转化与发出请求。抽象State类实现context的属性化访问与修改，两个具体类Cannon_State和Hammer_State分别表示杰斯的炮形态与锤形态。

In [8]:
from __future__ import annotations
from abc import ABC, abstractmethod

class Context:

    _state = None


    def __init__(self, state: State) -> None:
        self._state = state
        self._state.context = self
        print(f"杰斯: 初始形态为 {type(state).__name__}")

    def transition_to(self, state: State):
    

        print(f"杰斯: 切换状态为 {type(state).__name__}",end=" ")
        self._state = state
        self._state.context = self


    def request1(self):
        self._state.handle1()

    def request2(self):
        self._state.handle2()

class State(ABC):

    @property
    def context(self) -> Context:
        return self._context

    @context.setter
    def context(self, context: Context) -> None:
        self._context = context

    @abstractmethod
    def handle1(self) -> None:
        pass

    @abstractmethod
    def handle2(self) -> None:
        pass

class Cannon_State(State):

    def handle1(self) -> None:
        print("使用当前状态技能：冲击波")

    def handle2(self) -> None:
        self.context.transition_to(Hammer_State())
        print("增加10点护甲")

class Hammer_State(State):

    def handle1(self) -> None:
        print("使用当前状态技能：苍穹之跃")

    def handle2(self) -> None:
        self.context.transition_to(Cannon_State())
        print("增加10点攻击力")

if __name__ == "__main__":
    context = Context(Cannon_State())
    context.request1()
    context.request2()

    context.request1()
    context.request2()
    

杰斯: 初始形态为 Cannon_State
使用当前状态技能：冲击波
杰斯: 切换状态为 Hammer_State 增加10点护甲
使用当前状态技能：苍穹之跃
杰斯: 切换状态为 Cannon_State 增加10点攻击力
